In [36]:
import pandas as pd
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
import numpy as np
from PIL import Image

In [2]:
REAL = 'genuine_bin'
FORGED = 'forged_bin'

In [15]:
true = pd.read_csv('true_signatures.csv')
forged = pd.read_csv('forged_signatures.csv')

In [16]:
true.drop(true.columns[[0]], axis=1, inplace=True)
forged.drop(forged.columns[[0]], axis=1, inplace=True)

In [18]:
forged.columns = ['owner',	'fake', 'original', 'fake_img', 'original_img', 'target']
forged

,owner,fake,original,fake_img,original_img,target
0,14,1,1,NFI-00101014.png,NFI-01401014.png,0
1,14,1,2,NFI-00101014.png,NFI-01402014.png,0
2,14,1,3,NFI-00101014.png,NFI-01403014.png,0
3,14,1,4,NFI-00101014.png,NFI-01404014.png,0
4,14,1,5,NFI-00101014.png,NFI-01405014.png,0
...,...,...,...,...,...,...
745,4,5,1,NFI-08805004.png,NFI-00401004.png,0
746,4,5,2,NFI-08805004.png,NFI-00402004.png,0
747,4,5,3,NFI-08805004.png,NFI-00403004.png,0
748,4,5,4,NFI-08805004.png,NFI-00404004.png,0


In [36]:
real

,owner,sig1,sig2,sig1_img,sig2_img,target
0,1,1,2,NFI-00101001.png,NFI-00102001.png,1
1,1,1,3,NFI-00101001.png,NFI-00103001.png,1
2,1,1,4,NFI-00101001.png,NFI-00104001.png,1
3,1,1,5,NFI-00101001.png,NFI-00105001.png,1
4,1,2,3,NFI-00102001.png,NFI-00103001.png,1
...,...,...,...,...,...,...
295,30,2,4,NFI-03002030.PNG,NFI-03004030.PNG,1
296,30,2,5,NFI-03002030.PNG,NFI-03005030.PNG,1
297,30,3,4,NFI-03003030.PNG,NFI-03004030.PNG,1
298,30,3,5,NFI-03003030.PNG,NFI-03005030.PNG,1


In [31]:
forged = forged.groupby(['owner','fake']).head(3).reset_index(drop=True)

In [34]:
forged['fake_img'] = forged['fake_img'].apply(lambda x: FORGED+'/'+x)
forged['original_img'] = forged['original_img'].apply(lambda x: REAL+'/'+x)
forged

,owner,fake,original,fake_img,original_img,target
0,14,1,1,forged_bin/NFI-00101014.png,genuine_bin/NFI-01401014.png,0
1,14,1,2,forged_bin/NFI-00101014.png,genuine_bin/NFI-01402014.png,0
2,14,1,3,forged_bin/NFI-00101014.png,genuine_bin/NFI-01403014.png,0
3,14,2,1,forged_bin/NFI-00102014.png,genuine_bin/NFI-01401014.png,0
4,14,2,2,forged_bin/NFI-00102014.png,genuine_bin/NFI-01402014.png,0
...,...,...,...,...,...,...
442,4,4,2,forged_bin/NFI-08804004.png,genuine_bin/NFI-00402004.png,0
443,4,4,3,forged_bin/NFI-08804004.png,genuine_bin/NFI-00403004.png,0
444,4,5,1,forged_bin/NFI-08805004.png,genuine_bin/NFI-00401004.png,0
445,4,5,2,forged_bin/NFI-08805004.png,genuine_bin/NFI-00402004.png,0


In [ ]:
forged['fake_img'] = forged['fake_img'].apply(lambda x: FORGED+'/'+x)
forged['original_img'] = forged['original_img'].apply(lambda x: REAL+'/'+x)
forged

In [37]:
real['sig1_img'] = real['sig1_img'].apply(lambda x: REAL+'/'+x)
real['sig2_img'] = real['sig2_img'].apply(lambda x: REAL+'/'+x)
real

,owner,sig1,sig2,sig1_img,sig2_img,target
0,1,1,2,genuine_bin/NFI-00101001.png,genuine_bin/NFI-00102001.png,1
1,1,1,3,genuine_bin/NFI-00101001.png,genuine_bin/NFI-00103001.png,1
2,1,1,4,genuine_bin/NFI-00101001.png,genuine_bin/NFI-00104001.png,1
3,1,1,5,genuine_bin/NFI-00101001.png,genuine_bin/NFI-00105001.png,1
4,1,2,3,genuine_bin/NFI-00102001.png,genuine_bin/NFI-00103001.png,1
...,...,...,...,...,...,...
295,30,2,4,genuine_bin/NFI-03002030.PNG,genuine_bin/NFI-03004030.PNG,1
296,30,2,5,genuine_bin/NFI-03002030.PNG,genuine_bin/NFI-03005030.PNG,1
297,30,3,4,genuine_bin/NFI-03003030.PNG,genuine_bin/NFI-03004030.PNG,1
298,30,3,5,genuine_bin/NFI-03003030.PNG,genuine_bin/NFI-03005030.PNG,1


In [40]:
df1 = forged[['owner','fake_img', 'original_img','target']]
df2 = real[['owner','sig1_img', 'sig2_img','target']]

In [41]:
cols = ['owner','img1', 'img2','target']
df1.columns = cols
df2.columns = cols

In [44]:
dataset = pd.concat([df1, df2])

In [48]:
dataset.to_csv('train.csv')

In [46]:
dataset = dataset.sample(frac=1).reset_index(drop=True)
dataset

,owner,img1,img2,target
0,3,genuine_bin/NFI-00304003.png,genuine_bin/NFI-00305003.png,1
1,27,forged_bin/NFI-00101027.PNG,genuine_bin/NFI-02702027.PNG,0
2,27,forged_bin/NFI-00104027.PNG,genuine_bin/NFI-02703027.PNG,0
3,26,genuine_bin/NFI-02601026.PNG,genuine_bin/NFI-02604026.PNG,1
4,1,forged_bin/NFI-00302001.png,genuine_bin/NFI-00101001.png,0
...,...,...,...,...
742,11,forged_bin/NFI-00704011.png,genuine_bin/NFI-01101011.png,0
743,26,forged_bin/NFI-00105026.PNG,genuine_bin/NFI-02603026.PNG,0
744,3,forged_bin/NFI-00404003.png,genuine_bin/NFI-00303003.png,0
745,4,forged_bin/NFI-08802004.png,genuine_bin/NFI-00402004.png,0


In [4]:
# dataset['target'] = dataset['target'].astype('int16')
dataset = pd.read_csv('train.csv')
dataset.drop(dataset.columns[[0]], axis=1, inplace=True)

In [15]:
L = len(dataset)
L = int(L*0.7)
train_df = dataset.iloc[:L]
valid_df = dataset.iloc[L:]

,owner,img1,img2,target
522,6,genuine_bin/NFI-00601006.png,genuine_bin/NFI-00604006.png,1
523,14,forged_bin/NFI-00105014.png,genuine_bin/NFI-01403014.png,0
524,21,genuine_bin/NFI-02102021.png,genuine_bin/NFI-02103021.png,1
525,14,genuine_bin/NFI-01404014.png,genuine_bin/NFI-01405014.png,1
526,3,genuine_bin/NFI-00301003.png,genuine_bin/NFI-00305003.png,1
...,...,...,...,...
742,11,forged_bin/NFI-00704011.png,genuine_bin/NFI-01101011.png,0
743,26,forged_bin/NFI-00105026.PNG,genuine_bin/NFI-02603026.PNG,0
744,3,forged_bin/NFI-00404003.png,genuine_bin/NFI-00303003.png,0
745,4,forged_bin/NFI-08802004.png,genuine_bin/NFI-00402004.png,0


In [38]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        row = self.dataframe.iloc[index]
        img1 = torchvision.transforms.functional.to_tensor(Image.open(row["img1"]).convert('L'))
        img2 = torchvision.transforms.functional.to_tensor(Image.open(row["img2"]).convert('L'))
        target = row["target"]
        y = torch.from_numpy(np.array([target], dtype=np.float32))
        return (img1, img2, y)


In [13]:
BATCH = 16

In [40]:
train_dataset = Dataset(train_df)
train_loader = DataLoader(train_dataset, batch_size=BATCH, num_workers=0, pin_memory=False,)

valid_dataset = Dataset(valid_df)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH, shuffle=False,
    num_workers=0, pin_memory=False,)

In [51]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        # Setting up the Sequential of CNN Layers
        self.cnn1 = nn.Sequential( 
        nn.Conv2d(1, 96, kernel_size=11,stride=1),
        nn.ReLU(inplace=True),
        nn.LocalResponseNorm(5,alpha=0.0001,beta=0.75,k=2),
        nn.MaxPool2d(3, stride=2),

        nn.Conv2d(96, 256, kernel_size=5,stride=1,padding=2),
        nn.ReLU(inplace=True),
        nn.LocalResponseNorm(5,alpha=0.0001,beta=0.75,k=2),
        nn.MaxPool2d(3, stride=2),
        nn.Dropout2d(p=0.3),

        nn.Conv2d(256,384 , kernel_size=3,stride=1,padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(384,256 , kernel_size=3,stride=1,padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(3, stride=2),
        nn.Dropout2d(p=0.3),
        )

        # Defining the fully connected layers
        self.fc1 = nn.Sequential(
        # First Dense Layer
        nn.Linear(264704, 1024),
        nn.ReLU(inplace=True),
        nn.Dropout2d(p=0.5),
        # Second Dense Layer
        nn.Linear(1024, 128),
        nn.ReLU(inplace=True),
        # Final Dense Layer
        nn.Linear(128,2))

    def forward_once(self, x):
        # Forward pass 
        output = self.cnn1(x)
        print(output.size())
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output

    def forward(self, input1, input2):
        # forward pass of input 1
        output1 = self.forward_once(input1)
        # forward pass of input 2
        output2 = self.forward_once(input2)
        # returning the feature vectors of two inputs
        return output1, output2

In [23]:
class ContrastiveLoss(nn.Module):

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        # Find the pairwise distance or eucledian distance of two output feature vectors
        euclidean_distance = F.pairwise_distance(output1, output2)
        # perform contrastive loss calculation with the distance
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
        (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))

        return loss_contrastive

In [52]:
net = SiameseNetwork()
criterion = ContrastiveLoss()
optimizer = optim.Adam(net.parameters(),lr = 0.0005 )

In [53]:
counter = []
loss_history = [] 
iteration_number= 0
EPOCHS=5

In [54]:
for epoch in range(0,EPOCHS):
    for i, data in enumerate(train_loader,0):
        img0, img1 , label = data
        img0, img1 , label = img0, img1 , label
        optimizer.zero_grad()
        output1,output2 = net(img0,img1)
        loss_contrastive = criterion(output1,output2,label)
        loss_contrastive.backward()
        optimizer.step()
        if i %10 == 0 :
            print("Epoch number {}\n Current loss {}\n".format(epoch,loss_contrastive.item()))
            iteration_number +=10
            counter.append(iteration_number)
            loss_history.append(loss_contrastive.item())
show_plot(counter,loss_history)

torch.Size([16, 256, 22, 47])
torch.Size([16, 256, 22, 47])
Epoch number 0
 Current loss 1.2455577850341797

torch.Size([16, 256, 22, 47])
torch.Size([16, 256, 22, 47])
torch.Size([16, 256, 22, 47])
torch.Size([16, 256, 22, 47])
torch.Size([16, 256, 22, 47])
torch.Size([16, 256, 22, 47])
torch.Size([16, 256, 22, 47])
torch.Size([16, 256, 22, 47])
torch.Size([16, 256, 22, 47])
torch.Size([16, 256, 22, 47])
torch.Size([16, 256, 22, 47])
torch.Size([16, 256, 22, 47])
torch.Size([16, 256, 22, 47])
torch.Size([16, 256, 22, 47])
torch.Size([16, 256, 22, 47])
torch.Size([16, 256, 22, 47])


KeyboardInterrupt: 